In [1]:
import os
import pandas as pd
from re import sub
from tqdm.notebook import tqdm
from functions import readSet
tqdm.pandas()

In [2]:
datasets = os.listdir('./data')
postags  = readSet('./wordlists/postags.txt')

In [3]:
def cleanColumnName(name):
    name = sub(r'[^a-zA-Z]', '', name)
    name = name.lower()
    return name

In [4]:
for dataset in tqdm(datasets, desc="Datasets"):
    dataFile       = f'./data/{dataset}/Data-Cleaned.csv'
    dimensionsFile = f'./data/{dataset}/Dimensions.csv'
    predictionFile = f'./data/{dataset}/Word2Vec-Prediction.csv'
    outputFile     = f'./data/{dataset}/Stata-{dataset}.csv'

    if not os.path.exists(dataFile):
        raise ValueError(f"Dataset {dataset} has not been cleaned")
    if not os.path.exists(dimensionsFile):
        raise ValueError(f"Dimensions for {dataset} has not been calculated")
    if not os.path.exists(predictionFile):
        raise ValueError(f"Dataset {dataset} has not been predicted")

    if not os.path.exists(outputFile):
        data       = pd.read_csv(dataFile)
        dimensions = pd.read_csv(dimensionsFile)
        prediction = pd.read_csv(predictionFile)
        data = data[['text','sentiment']]
        prediction = prediction['predicted']
        dimensions = dimensions.drop(columns=list(postags))
        if 'words' in dimensions:
            dimensions = dimensions.drop(columns=['words', 'sentences'])

        output = pd.concat([data, prediction, dimensions], axis=1)
        output.columns = output.columns.map(cleanColumnName)
        output.to_csv(outputFile, index=False)

Datasets:   0%|          | 0/3 [00:00<?, ?it/s]